In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
from OpenGL.GLUT import GLUT_BITMAP_HELVETICA_18

import sys
import math

# Camera control
camera_angle = [0, 0]
camera_pos = [0, 5, 20]

# Flag positions
red_flag_pos = [-10, 0, -10]
blue_flag_pos = [10, 0, 10]
# player positions
player_pos = [0.0, 0.0, 0.0]
player_speed = 0.5
key_states = {b"w": False, b"s": False, b"a": False, b"d": False}
player_health = 5
# Direction player is facing: [dx, dy, dz]
player_direction = [0, 0, 1]  # Initially facing +Z (forward)


player_has_flag = False

player_wins = False

fireballs = []  # List of fireballs, each is a dict with position and velocity
fireball_speed = 1.0

tower_pos = [10.0, 0.0, 7.0]  # Near the blue flag
tower_fireballs = []
tower_fireball_speed = 0.5
tower_cooldown = 0  # Countdown until next fire
tower_alerted = False


def init():
    glClearColor(0.5, 0.8, 1.0, 1.0)  # Sky blue background
    glEnable(GL_DEPTH_TEST)
    glShadeModel(GL_SMOOTH)
    glutKeyboardFunc(key_down)
    glutKeyboardUpFunc(key_up)
    glutMouseFunc(mouse_click)

def draw_text(x, y, text, font=GLUT_BITMAP_HELVETICA_18):
    glColor3f(1, 1, 1)
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, 800, 0, 600)
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadIdentity()
    glRasterPos2f(x, y)
    for ch in text:
        glutBitmapCharacter(font, ord(ch))
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)


def reshape(w, h):
    glViewport(0, 0, w, h)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(60.0, float(w)/float(h), 1.0, 100.0)
    glMatrixMode(GL_MODELVIEW)

def draw_flag(position, color):
    glPushMatrix()
    glTranslatef(*position)
    glColor3fv(color)
    glutSolidCube(1.0)
    glPopMatrix()

def draw_base(position, color):
    glPushMatrix()
    glTranslatef(*position)
    glScalef(4, 0.2, 4)
    glColor3fv(color)
    glutSolidCube(1.0)
    glPopMatrix()

# Assume this is your shared ground render function
def render_ground():
    glColor3f(0.3, 0.6, 0.3)  # Grass green
    glBegin(GL_QUADS)
    glVertex3f(-50, -0.1, -50)
    glVertex3f(-50, -0.1, 50)
    glVertex3f(50, -0.1, 50)
    glVertex3f(50, -0.1, -50)
    glEnd()









def display():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    update_player()
    check_flag_pickup()
    update_fireballs()
    

    # Set camera
    gluLookAt(camera_pos[0], camera_pos[1], camera_pos[2],
              0, 0, 0,
              0, 1, 0)

    # Draw terrain
    render_ground()

    # Draw bases
    draw_base([-10, 0, -10], (1, 0, 0))  # Red base
    draw_base([10, 0, 10], (0, 0, 1))    # Blue base

    # Draw flags
    if not player_has_flag:
        draw_flag(blue_flag_pos, (0, 0, 1))
    draw_flag(red_flag_pos, (1, 0, 0))

    # Draw player
    glPushMatrix()
    glTranslatef(*player_pos)
    glColor3f(1, 1, 0)  # Yellow
    glutSolidCube(1.0)
    glPopMatrix()

    if player_wins:
        draw_text(300, 300, "🏁 YOU WIN!",GLUT_BITMAP_HELVETICA_18)

    # Draw fireballs
    for fb in fireballs:
        glPushMatrix()
        glTranslatef(*fb["pos"])
        glColor3f(1, 0.5, 0)  # Orange
        glutSolidSphere(0.3, 10, 10)
        glPopMatrix()

    # Draw the tower
    glPushMatrix()
    glTranslatef(*tower_pos)
    glColor3f(0.3, 0.3, 0.3)  # Gray
    glScalef(1, 3, 1)
    glutSolidCube(1.0)
    glPopMatrix()

    # Draw tower fireballs
    for fb in tower_fireballs:
        glPushMatrix()
        glTranslatef(*fb["pos"])
        glColor3f(1.0, 0.1, 0.1)  # Red fireball
        glutSolidSphere(0.3, 10, 10)
        glPopMatrix()
        # Show player health in top-left corner
    draw_text(10, 570, f"Health: {player_health}", font=GLUT_BITMAP_HELVETICA_18, color=(1, 0, 0))
    
    glutSwapBuffers()

def key_down(key, x, y):
    if key in key_states:
        key_states[key] = True
    elif key == b' ':  # Spacebar
        shoot_fireball()

def key_up(key, x, y):
    if key in key_states:
        key_states[key] = False

def mouse_click(button, state, x, y):
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN:
       
        reset_game()


def update_player():
    global player_direction

    if key_states[b"w"]:
        player_pos[2] -= player_speed
        player_direction = [0, 0, -1]
    if key_states[b"s"]:
        player_pos[2] += player_speed
        player_direction = [0, 0, 1]
    if key_states[b"a"]:
        player_pos[0] -= player_speed
        player_direction = [-1, 0, 0]
    if key_states[b"d"]:
        player_pos[0] += player_speed
        player_direction = [1, 0, 0]

def distance(p1, p2):
    return math.sqrt(sum([(p1[i] - p2[i])**2 for i in range(3)]))




def shoot_fireball():
    fireball = {
        "pos": player_pos[:],       # copy current position
        "vel": player_direction[:]  # copy current facing direction
    }
    fireballs.append(fireball)


def update_fireballs():
    global fireballs, tower_alerted

    new_fireballs = []
    for fb in fireballs:
        # Update position
        fb["pos"] = [fb["pos"][i] + fb["vel"][i] * fireball_speed for i in range(3)]

        # Check for tower collision
        if distance(fb["pos"], tower_pos) < 1.5:
            tower_alerted = True
            print("🚨 Tower alerted by player's fireball!")
            continue  # Remove this fireball after hitting

        # Keep fireball in scene bounds
        if -50 < fb["pos"][0] < 50 and -50 < fb["pos"][2] < 50:
            new_fireballs.append(fb)

    fireballs = new_fireballs






def shoot_tower_fireball():
    dir_vec = [player_pos[i] - tower_pos[i] for i in range(3)]
    magnitude = math.sqrt(sum([x**2 for x in dir_vec]))
    if magnitude == 0:
        return
    velocity = [x / magnitude for x in dir_vec]
    fireball = {
        "pos": tower_pos[:],
        "vel": velocity
    }
    tower_fireballs.append(fireball)





def main():
    glutInit(sys.argv)
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
    glutInitWindowSize(800, 600)
    glutCreateWindow(b"Capture the Flag 3D Environment")
    init()
    glutDisplayFunc(display)
    glutReshapeFunc(reshape)
    glutIdleFunc(glutPostRedisplay)
    glutMainLoop()

if __name__ == '__main__':
    main()


🚩 Blue Flag Captured!
Checking hit: Fireball at [9.878732187481834, 0.0, 7.485071250072666], Player at [9.0, 0.0, 11.0], Distance = 3.62
Checking hit: Fireball at [9.757464374963668, 0.0, 7.970142500145332], Player at [9.5, 0.0, 11.0], Distance = 3.04
Checking hit: Fireball at [9.636196562445502, 0.0, 8.455213750217998], Player at [10.0, 0.0, 11.0], Distance = 2.57
Checking hit: Fireball at [9.514928749927336, 0.0, 8.940285000290665], Player at [10.5, 0.0, 11.0], Distance = 2.28
Checking hit: Fireball at [9.39366093740917, 0.0, 9.42535625036333], Player at [11.0, 0.0, 11.0], Distance = 2.25
Checking hit: Fireball at [9.272393124891003, 0.0, 9.910427500435997], Player at [11.5, 0.0, 11.0], Distance = 2.48
Checking hit: Fireball at [9.151125312372837, 0.0, 10.395498750508663], Player at [12.0, 0.0, 11.0], Distance = 2.91
Checking hit: Fireball at [9.029857499854671, 0.0, 10.88057000058133], Player at [12.0, 0.0, 11.0], Distance = 2.97
Checking hit: Fireball at [8.908589687336505, 0.0, 11

Exception ignored on calling ctypes callback function: <function GLUTCallback.__call__.<locals>.safeCall at 0x0000024C2DE99800>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\OpenGL\GLUT\special.py", line 130, in safeCall
    return function( *args, **named )
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_9744\1100303211.py", line 207, in display
KeyboardInterrupt: 


: 